<a href="https://colab.research.google.com/github/pabodiniudari/GEE-Classification/blob/main/206123F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First we will import the earth engine package
import ee
import folium

In [ ]:
# Trigger the authentication
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=IF5tPgczG-d9IAKUCIBHS_U4G207Ji9pIaiVjD2fQVs&tc=TCQyRBfDmLvKYE6ohkGdGGI2DHI5C2XGJ6-SErgb254&cc=WLvnJ7gCuB7IkpfVWVM8TVv0K9u3q3tVzFdfnHgHspY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnPn77vh6DbR8t1TcZ7SZPF67BvmAT-9r7DUYOPDpvowdTAJOsrZoA

Successfully saved authorization token.


In [ ]:
# Initialize the library
ee.Initialize()

In [ ]:
# Import the feature collection as the world dataset
world = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

In [ ]:
# Map over the feature collection to extract the ’country_na’ attribute and get unique values
country_names = world.aggregate_array('country_na').distinct().getInfo()

In [ ]:
# Print the list of country names
for country in country_names:
  print(country)

Chad
Malawi
Zambia
Zimbabwe
Botswana
Namibia
Angola
Burundi
Rwanda
South Africa
Lesotho
Swaziland
Mayotte
Niger
Sudan
Libya
Koualou Area
Tunisia
Egypt
Bir Tawil
Halaib Triangle
Eritrea
Ethiopia
Djibouti
Somalia
South Sudan
Abyei Area
Kenya
Uganda
Tanzania
Mozambique
Comoros
Madagascar
Cabo Verde
Western Sahara
Mauritania
Morocco
Spain (Canary Is)
Portugal (Madeira Is)
Spain (Africa)
Algeria
Mali
Burkina Faso
Togo
Ghana
Cote d'Ivoire
Guinea
Senegal
Gambia, The
Guinea-Bissau
Sierra Leone
Liberia
Benin
Nigeria
Cameroon
Equatorial Guinea
Sao Tome & Principe
Gabon
Rep of the Congo
Central African Rep
Dem Rep of the Congo
Hong Kong
Macau
Taiwan
Senkakus
Japan
Korea, South
Korea, North
Korean Is. (UN Jurisdiction)
China
Liancourt Rocks
Russia
Ukraine
Romania
Moldova
Finland
Norway
Andorra
Italy
Monaco
Malta
San Marino
Vatican City
Croatia
Albania
Bosnia & Herzegovina
Montenegro
Serbia
Greece
Macedonia
Kosovo
Bulgaria
Portugal (Azores)
Portugal
Gibraltar
Spain
France
Svalbard
Sweden
Denmark
Es

In [ ]:
country = 'Brazil'
aoi = world.filter(ee.Filter.eq('country_na', country))
years = list(range(2010,2016))
NDVI_img = {}
for year in years:
  str_year = str(year)
  image = (ee.ImageCollection("MODIS/006/MODO9Q1")
  .select(["sur_refl_b01" , "sur_refl_b02"])
  .filterDate(f"{str_year}-01-01" , f"{str_year}-12-31")
  .filterBounds(aoi))
  mean_NDVI = image.map(lambda img:img.normalizedDifference(["sur_refl_b01" , "sur_refl_b02"])).mean()
  NDVI_img[year] = mean_NDVI

In [ ]:
# Define a colour palette to visualize the images
RF_PALETTE = "FFFFFF,CE7E45,DF923D,F1B555,FCD163,99B718,74A901,66A000,529400,3E8601,207401,056201,004C00,023B01,012E01,011D01,011301"
vis_params = {
    'min': -1,
    'max': 1,
    'palette': RF_PALETTE
}

In [ ]:
# Create a map centered around the country
location = [-14.2350, -51.9253]
map_country = folium.Map(location=location, zoom_start=7)



In [ ]:
# Function to visualize Earth Engine image layers using folium.
def add_ee_layer(self, ee_object, vis_params, name):
    try:
        # Display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # Display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # Display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
                data=ee_object.getInfo(),
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # Display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
    except:
        print("Could not display {}".format(name))


In [ ]:
# Add custom method to folium
folium.Map.add_ee_layer = add_ee_layer


In [ ]:
for year in years:
  NDVI = NDVI_img[year]
  map_country.add_ee_layer(NDVI.clip(aoi),vis_params, f"NDVI{year}")


Could not display NDVI2010
Could not display NDVI2011
Could not display NDVI2012
Could not display NDVI2013
Could not display NDVI2014
Could not display NDVI2015


In [ ]:
map_country.add_ee_layer(aoi, {}, country)

In [ ]:
folium.LayerControl().add_to(map_country)

In [ ]:
# Display the map
display(map_country)


In [ ]:
# Define the years for which you want to get the precip images
years = [2017, 2018, 2019]
# Initialize an empty dictionary to store the precip images
precip_images = {}
# Loop through each year
for year in years:
  # Convert the year to string for date formatting
  str_year = str(year)
  # Load an image of daily precipitation in mm/day.
  # Sums up the precipitation values from all the images in the filtered ImageCollection.
  # The result is a single ee.Image where each pixel value represents
  # the total accumulated precipitation for the entire year within the AOI.
  precip = (ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
  .select('precipitation')
  .filterDate(f'{str_year}-01-01', f'{str_year}-12-31')
  .filterBounds(aoi)
  .sum())
  # Store the precip image in the dictionary using the year as the key
  precip_images[year] = precip
  # Add the precipitation layers for each year
for year in [2017, 2018, 2019]:
  precip = precip_images[year]
  map_country.add_ee_layer(precip.clip(aoi), vis_params, f'Rainfall {year}')
  # Add the AOI layer
  map_country.add_ee_layer(aoi, {}, country)
  # Add Layer Control
  folium.LayerControl().add_to(map_country)
  # Display the map
  display(map_country)

In [ ]:
# Define the years for which you want to get the NDVI images
years = [2017, 2018, 2019]

In [ ]:
# Initialize an empty dictionary to store the NDVI images
ndvi_images = {}

In [ ]:
# Loop through each year
for year in years:
  # Convert the year to string for date formatting
  str_year = str(year)
  # Load the image collection for the given year within the AOI.
  # The MODIS MOD09Q1 product has a band named ’sur_refl_b02’
  # for red and ’sur_refl_b01’ for NIR.
  images = (ee.ImageCollection('MODIS/006/MOD09Q1')
    .select(['sur_refl_b01', 'sur_refl_b02'])
    .filterDate(f'{str_year}-01-01', f'{str_year}-12-31')
    .filterBounds(aoi))
  # Compute the mean NDVI for the entire year.
  # NDVI formula: (NIR - RED) / (NIR + RED)
  mean_ndvi = images.map(lambda img:
                         img.normalizedDifference(['sur_refl_b01',
                                                   'sur_refl_b02'])).mean()
  # Store the mean NDVI image in the dictionary using the year as the key
  ndvi_images[year] = mean_ndvi
  # Now ndvi_images contains the mean NDVI images for 2017, 2018, and 2019

In [ ]:
# Add custom method to folium
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Define a colour palette to visualize the images
RF_PALETTE = "ff0000, ffff00, 00ffff, 0000ff"
vis_params = {
'min': -1,
'max': 1,
'palette': RF_PALETTE
}

In [ ]:
# Create a map centered around the country
location = [6.9271, 80.8612]
map_country = folium.Map(location=location, zoom_start=7)


In [ ]:
# Add the ndvi layers for each year
for year in [2017, 2018, 2019]:
  ndvi = ndvi_images[year]
  map_country.add_ee_layer(ndvi.clip(aoi), vis_params, f'NDVI {year}')

In [ ]:
# Add the AOI layer
map_country.add_ee_layer(aoi, {}, country)

In [ ]:
# Add Layer Control
folium.LayerControl().add_to(map_country)

In [ ]:
# Display the map
display(map_country)

In [ ]:
# Define the selected country
country = 'Brazil'

# Filter the feature collection for the selected country
aoi = world.filter(ee.Filter.eq('country_na', country))

# Define the years for which you want to get the NDVI images
years = list(range(2010, 2016))

# Initialize an empty dictionary to store the NDVI images
ndvi_images = {}

# Define visualization parameters
NDVI_PALETTE = "FFFFFF,CE7E45,DF923D,F1B555,FCD163,99B718,74A901,66A000,529400,3E8601,207401,056201,004C00,023B01,012E01,011D01,011301"

vis_params = {
    'min': -1,
    'max': 1,
    'palette': NDVI_PALETTE
}

# Create a map centered around the selected country
location = [-14.2350, -51.9253]

map_country = folium.Map(location=location, zoom_start=7)

# Add the NDVI layers for each year
for year in years:
    if year in ndvi_images:
        NDVI = ndvi_images[year]
        map_country.add_ee_layer(NDVI.clip(aoi), vis_params, f'NDVI {year}')
    else:
        print(f"NDVI data not found for {year}")

# Add the AOI layer
map_country.add_ee_layer(aoi, {}, country)

# Add Layer Control
folium.LayerControl().add_to(map_country)

# Display the map
display(map_country)


NDVI data not found for 2010
NDVI data not found for 2011
NDVI data not found for 2012
NDVI data not found for 2013
NDVI data not found for 2014
NDVI data not found for 2015
